In [18]:
import pandas as pd
import dateutil.parser

In [22]:
data = pd.read_csv(".data/stock_data.csv").drop(columns=["Unnamed: 0"])
#data = data.reset_index()
data["Date"] = data["Date"].apply(lambda x: dateutil.parser.parse(x))
data["Date"] = data["Date"].dt.date
data['week'] = data['Date'].apply(lambda x: x.isocalendar()[1])
data['year'] = data['Date'].apply(lambda x: x.year)
data['day_of_week'] = data['Date'].apply(lambda x: x.isocalendar()[2])
data = data.set_index("Date")
data

,Open,High,Low,Close,Volume,Dividends,Stock Splits,week,year,day_of_week
Date,,,,,,,,,,
2019-02-14,40.844916,41.217959,40.765492,41.107250,87342800,0.00,0.0,7,2019,4
2019-02-15,41.215546,41.323849,40.854534,41.015785,98507200,0.00,0.0,7,2019,5
2019-02-19,40.844911,41.261277,40.791962,41.138531,75891200,0.00,0.0,8,2019,2
2019-02-20,41.201105,41.713743,41.152971,41.403271,104457600,0.00,0.0,8,2019,3
2019-02-21,41.347916,41.485098,40.986904,41.169815,68998800,0.00,0.0,8,2019,4
...,...,...,...,...,...,...,...,...,...,...
2024-02-07,190.397053,190.806534,188.369641,189.168625,53439000,0.00,0.0,6,2024,3
2024-02-08,189.148646,189.298448,187.111252,188.080017,40962000,0.00,0.0,6,2024,4
2024-02-09,188.649994,189.990005,188.000000,188.850006,45099900,0.24,0.0,6,2024,5


In [23]:
daily_open_close = data.groupby(by=["year","week","day_of_week"])[["Open","Close"]].mean()
open_close = daily_open_close.reset_index("day_of_week")
grouped = open_close.groupby(by=["year","week"])
open_close["start_of_week"] = grouped[["day_of_week"]].min()
open_close["end_of_week"] = grouped[["day_of_week"]].max()
open_close

day_of_week        Open       Close  start_of_week  end_of_week
year week                                                                 
2019 1               1   70.410440   70.911530              1            2
     1               2   70.524783   71.429665              1            2
     7               4   40.844916   41.107250              4            5
     7               5   41.215546   41.015785              4            5
     8               2   40.844911   41.138531              2            5
...                ...         ...         ...            ...          ...
2024 6               3  190.397053  189.168625              1            5
     6               4  189.148646  188.080017              1            5
     6               5  188.649994  188.850006              1            5
     7               1  188.419998  187.149994              1            2
     7               2  185.770004  185.039993              1            2

[1258 rows x 5 columns]

In [24]:
unique_weeks = open_close.reset_index()[["year","week","start_of_week","end_of_week"]].drop_duplicates()
weekly_opens = unique_weeks.rename(columns = {"start_of_week":"day_of_week"}).set_index(["year","week","day_of_week"]).join(daily_open_close["Open"]).drop(columns = ["end_of_week"]).reset_index().set_index(["year","week"]).drop(columns="day_of_week")
weekly_closes = unique_weeks.rename(columns = {"end_of_week":"day_of_week"}).set_index(["year","week","day_of_week"]).join(daily_open_close["Close"]).drop(columns = ["start_of_week"]).reset_index().set_index(["year","week"]).drop(columns="day_of_week")
unique_weeks = unique_weeks.set_index(["year","week"])
unique_weeks["Open"] = weekly_opens
unique_weeks["Close"] = weekly_closes
unique_weeks.drop(columns=["start_of_week","end_of_week"], inplace=True)
unique_weeks

Open       Close
year week                        
2019 1      70.410440   71.429665
     7      40.844916   41.015785
     8      40.844911   41.629505
     9      41.915913   42.110847
     10     42.284134   41.615067
...               ...         ...
2024 3     181.927853  191.315872
     4     192.054929  192.174774
     5     191.765299  185.613159
     6     187.910213  188.850006
     7     188.419998  185.039993

[263 rows x 2 columns]

In [39]:
news = pd.read_csv(".data/processed_news.csv")
news = news.drop(columns=["Unnamed: 0"])
news

,title,description,published date,url,publisher,fetched_title,fetched_body,week,year,gpt_results
0,Apple Watch Series 5 in (PRODUCT)RED Could Lau...,Apple Watch Series 5 in (PRODUCT)RED Could Lau...,2019-12-30 08:00:00+00:00,https://news.google.com/rss/articles/CBMiSmh0d...,"{'href': 'https://www.macrumors.com', 'title':...",Apple Watch Series 5 in (PRODUCT)RED Could Lau...,Apple may be planning to expand its Apple Watc...,1,2019,Summary: Apple may be planning to release a ne...
1,"Apple, Microsoft, Amazon, Alphabet now worth c...","Apple, Microsoft, Amazon, Alphabet now worth c...",2019-12-31 08:00:00+00:00,https://news.google.com/rss/articles/CBMiVGh0d...,"{'href': 'https://qz.com', 'title': 'Quartz'}",The four largest tech companies are now worth ...,"Last year, it was a big deal when Apple became...",1,2019,Summary: Apple's market capitalization has sur...
2,Apple to Sell Modified iPhone 7 and iPhone 8 i...,Apple to Sell Modified iPhone 7 and iPhone 8 i...,2019-02-14 08:00:00+00:00,https://news.google.com/rss/articles/CBMiTGh0d...,"{'href': 'https://www.macrumors.com', 'title':...",Apple to Sell Modified iPhone 7 and iPhone 8 i...,Apple today confirmed rumors that it will star...,7,2019,Summary: Apple is replacing Intel chips with Q...
3,Apple resumes selling certain iPhones in Germa...,Apple resumes selling certain iPhones in Germa...,2019-02-14 08:00:00+00:00,https://news.google.com/rss/articles/CBMiXGh0d...,"{'href': 'https://www.cnet.com', 'title': 'CNET'}",Apple resumes selling certain iPhones in Germa...,Apple's iPhone 7 and iPhone 8 will be on sale ...,7,2019,Summary: Apple will resume sales of iPhone 7 a...
4,Putting Apple Watch 4 to the Test in Afib - Me...,Putting Apple Watch 4 to the Test in Afib Med...,2019-02-24 08:00:00+00:00,https://news.google.com/rss/articles/CBMiOWh0d...,"{'href': 'https://www.medpagetoday.com', 'titl...",Putting Apple Watch 4 to the Test in Afib,"My first patient of the morning, a delightful ...",8,2019,Summary: A cardiologist evaluates the effectiv...
...,...,...,...,...,...,...,...,...,...,...
523,"Apple Vision Pro: How to Buy $3,499 Headset, A...","Apple Vision Pro: How to Buy $3,499 Headset, A...",2024-02-04 08:00:00+00:00,https://news.google.com/rss/articles/CBMihgFod...,"{'href': 'https://www.bloomberg.com', 'title':...",Are you a robot?,Why did this happen?\n\nPlease make sure your ...,5,2024,Summary: The news discusses a sudden drop in A...
524,Apple Vision Pro Review: The New Era of Spatia...,Apple Vision Pro Review: The New Era of Spatia...,2024-02-09 14:02:42+00:00,https://news.google.com/rss/articles/CBMiWWh0d...,"{'href': 'https://www.xrtoday.com', 'title': '...",Apple Vision Pro Review: The New Era of Spatia...,"In this Apple Vision Pro review, we’re finally...",6,2024,Summary: The Apple Vision Pro is a high-end XR...
525,Apple Engineers Allegedly Able to Use Vision P...,Apple Engineers Allegedly Able to Use Vision P...,2024-02-06 16:13:16+00:00,https://news.google.com/rss/articles/CBMiTmh0d...,"{'href': 'https://www.macrumors.com', 'title':...",Apple Engineers Allegedly Able to Use Vision P...,Apple's new Vision Pro headset can serve as an...,6,2024,Summary: Apple's new Vision Pro headset allows...
526,"Apple, Amazon, Microsoft, Meta and Nvidia are ...","Apple, Amazon, Microsoft, Meta and Nvidia are ...",2024-02-12 11:00:00+00:00,https://news.google.com/rss/articles/CBMiUGh0d...,"{'href': 'https://finance.yahoo.com', 'title':...","Apple, Amazon, Microsoft, Meta and Nvidia are ...","For Immediate Release\n\nChicago, IL – Februar...",7,2024,Summary: Zacks.com releases list of companies ...


In [47]:
joined_data = unique_weeks.join(news.groupby(by=["year","week"])[["gpt_results"]].sum())

In [59]:
joined_data

Open       Close  \
year week                           
2019 1      70.410440   71.429665   
     7      40.844916   41.015785   
     8      40.844911   41.629505   
     9      41.915913   42.110847   
     10     42.284134   41.615067   
...               ...         ...   
2024 3     181.927853  191.315872   
     4     192.054929  192.174774   
     5     191.765299  185.613159   
     6     187.910213  188.850006   
     7     188.419998  185.039993   

                                                 gpt_results  
year week                                                     
2019 1     Summary: Apple may be planning to release a ne...  
     7     Summary: Apple is replacing Intel chips with Q...  
     8     Summary: A cardiologist evaluates the effectiv...  
     9     Summary: Former Apple store employees reveal t...  
     10    Summary: Sen. Elizabeth Warren proposed breaki...  
...                                                      ...  
2024 3     Summary: Apple Vision Pro pre-orders have begu...  
     4     Summary: The M2 Mac Mini is currently availabl...  
     5     Summary: Apple's highly-anticipated Vision Pro...  
     6     Summary: The Apple Vision Pro is a high-end XR...  
     7     Summary: Zacks.com releases list of companies ...  

[263 rows x 3 columns]

In [68]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import langchain
gpt = ChatOpenAI(model="gpt-3.5-turbo")
company_profile = """Description: Apple Inc. (AAPL) is an American multinational technology company that specializes in consumer electronics, software, and services. It is best known for its iPhone smartphones, iPad tablets, Mac computers, and services like the App Store, Apple Music, and iCloud. Apple's emphasis on design and innovation has made it one of the most valuable companies in the world.

Positive Factors:
- **Innovation and Product Launches**: Continuous innovation and the release of new products can drive consumer interest and sales.
- **Brand Loyalty**: Strong brand recognition and a loyal customer base can sustain revenue and profit margins.
- **Services Growth**: Expansion of Apple's services sector, including streaming, financial services, and cloud storage, diversifies revenue streams.
- **Global Expansion**: Entry into emerging markets can provide new revenue sources.
- **Macroeconomic Trends**: A strong economic environment, with high consumer spending and low unemployment rates, can increase demand for Apple's products.

Negative Factors:
- **Competition**: Intense competition from other tech giants and emerging players can impact market share and pricing.
- **Supply Chain Disruptions**: Reliance on global supply chains makes Apple vulnerable to disruptions, tariffs, and geopolitical tensions.
- **Regulatory Challenges**: Increased scrutiny and regulation in key markets, including antitrust investigations and privacy regulations, can affect operations and costs.
- **Market Saturation**: High penetration rates in key markets can slow growth, making it challenging to maintain high sales volumes.
- **Macroeconomic Downturns**: Economic downturns, high inflation rates, or rising interest rates can reduce consumer spending on non-essential goods, impacting sales."""

news_summary_prompt = ChatPromptTemplate.from_template(f"""Instruction: Forecast next week stock return (price change) for symbol, given the company profile, historical weekly news summary, keywords, and stock returns, and optionally the examples from other stocks of a similar company. Return a single stock return.
The trend is represented by bins "D5+", "D5", "D4", "D3", "D2", "D1", "U1", "U2", "U3", "U4", "U5", "U5+", where "D5+" means price dropping more than 5%, D5 means price dropping between 4% and 5%, "D4" means price dropping between 3% and 4%, "U5+" means price rising more than 5%, "U5" means price rising between 4% and 5%, "D4" means price rising between 3% and 4%, etc. Company Profile: {company_profile}
Recent News: News are ordered from oldest news to latest news. {{news}}""")


news_summary_chain = news_summary_prompt | gpt | StrOutputParser()
gpt_results = news_summary_chain.invoke({"news": joined_data["gpt_results"].iloc[-1]})
print(gpt_results)

Based on the positive factors such as continuous innovation, brand loyalty, services growth, and global expansion, as well as the recent news of stability and resilience in the tech sector, it is likely that Apple Inc. (AAPL) stock will have a positive return next week. Therefore, the forecasted stock return is in the "U1" to "U3" range, indicating a price increase between 1% and 3%.
